In [1]:
import pandas as pd
import json 
import unicodedata
import numpy as np
import re
from collections import Counter

In [2]:

def remove_accents(input_str):
    nfkd_form = unicodedata.normalize('NFKD', input_str)
    return u"".join([c for c in nfkd_form if not unicodedata.combining(c)])


def remove_regex(regex: str, string: str) -> str:
    try: 
        find = re.search(regex, string).group()
        return string.replace(find, "")
    except: 
        return string

def levenshteinDistanceDP(token1, token2):
    distances = np.zeros((len(token1) + 1, len(token2) + 1))

    for t1 in range(len(token1) + 1):
        distances[t1][0] = t1

    for t2 in range(len(token2) + 1):
        distances[0][t2] = t2
        
    a = 0
    b = 0
    c = 0
    
    for t1 in range(1, len(token1) + 1):
        for t2 in range(1, len(token2) + 1):
            if (token1[t1-1] == token2[t2-1]):
                distances[t1][t2] = distances[t1 - 1][t2 - 1]
            else:
                a = distances[t1][t2 - 1]
                b = distances[t1 - 1][t2]
                c = distances[t1 - 1][t2 - 1]
                distances[t1][t2] = min(a,b,c) + 1 

    #printDistances(distances, len(token1), len(token2))
    return distances[len(token1)][len(token2)]

In [3]:
levenshteinDistanceDP("Rua Almirante", "Almirante")

4.0

In [4]:

estabelecimentos = pd.read_csv("../data/estabelecimentos/estabelecimentos-420000-202202.csv")
estabelecimentos["cnes"] = estabelecimentos.CNES
estabelecimentos.set_index("cnes",inplace=True)

# Estabelecimentos

# Dados do OSM

In [5]:
with open("../data/geoloc/logradouros.json") as fp:
    logradouros = json.load(fp)

In [6]:
# Essa função ficou demasiadamente ruim, não existe estrutura no displayname
def display_name_to_dict(display_name: str) -> dict:
    composicao  = ["nome", "n", "logradouro", "bairro", "municipio", 
    "imediata", "intermediaria", "estado", "região", "CEP", "pais"]
    array_endereco = [x.strip() for x in display_name.split(",")]
    nn = len(array_endereco)
    if nn < 8:
        # inferno de caso que não tem bairro
        composicao.remove("bairro")
    
    if nn == 12:
        # inferno de caso com distrito
        composicao.insert(3,"distrito")
        
    cep_regex = r"\d{5}-\d{3}"
    if not re.search(cep_regex, display_name):
        composicao.remove("CEP")

    dicionario = dict(zip(composicao[-nn:], array_endereco))
    if "CEP" in dicionario:
        dicionario["CEPhead"] = dicionario["CEP"][:5]
    else: 
        dicionario["CEP"] = ""
        dicionario["CEPhead"] = ""


    return dicionario


def logradouros_list(cnes : str) -> list:
    return [display_name_to_dict(x["display_name"]) for x in logradouros[cnes] ]


# CEP Database

In [7]:
cepdb = pd.read_csv("../data/geoloc/CEPdatabase.csv")
cepdb.set_index("cep",inplace=True)

In [8]:
def get_logradouro_cepdb(cnes: int):
    try: 
        return cepdb.loc[get_endereco(cnes)["CEP_n"],"logradouro"]
    except:
        return None

def logradouro_from_cep(cep: int) -> str:
    try:
        logradouro = cepdb.loc[cep,"logradouro"]
        if len(logradouro) == 0:
            return "not from cep"
        else:
            return logradouro
    except:
        return "not from cep"

def latitude_from_cep(cep: int) -> str:
    try:
        return float(cepdb.loc[cep,"latitude"])
    except:
        return 0.0
def longitude_from_cep(cep: int) -> str:
    try:
        return float(cepdb.loc[cep,"longitude"])
    except:
        return 0.0



estabelecimentos["logradouro_from_cep"] = estabelecimentos.CEP.apply(logradouro_from_cep)
estabelecimentos["lat_cep"] = estabelecimentos.CEP.apply(latitude_from_cep)
estabelecimentos["lon_cep"] = estabelecimentos.CEP.apply(longitude_from_cep)


In [9]:
def get_endereco(cnes: int) -> dict:
    """Função que pega um CNES e retorna um dicionário de endereço"""
    x = estabelecimentos.loc[cnes,["LOGRADOURO", "NUMERO", "BAIRRO", "MUNICIPIO", "IBGE", 'CEP', 'logradouro_from_cep']].values
    resultado = dict(zip( ["logradouro", "n", "bairro", "municipio", 'ibge', "CEP_n", 'cep_logr'], x))
    cep_string = str(resultado["CEP_n"])
    resultado["CEPhead"] = cep_string[:5]
    resultado["CEP"] = resultado["CEPhead"] + "-" + cep_string[5:]
    return resultado

In [10]:
POUCA_DIFERENCA = ['rua','avenida','rodovia','travessa', 'estrada','alameda','servidao',
'doutor','coronel', 'marechal', 'desembargador', 'engenheiro',
 'prefeito', 'vereador','governador','presidente',
 'quinze','sete', '15', 'de']

def compara_strings(str1: str, str2: str):
    lstr1, lstr2 = remove_accents(str(str1).lower()), remove_accents(str(str2).lower())
    N = min(len(lstr1),len(lstr2))
    if lstr1.strip() in lstr2: 
        return 1.0
    else: 
        return 1 - levenshteinDistanceDP(lstr1, lstr2)/N


In [11]:
compara = lambda x: compara_strings(x["LOGRADOURO"], x["logradouro_from_cep"] )

estabelecimentos["check_logradouro"] = estabelecimentos.apply(compara, axis=1)


In [12]:

estabelecimentos["n_osm_results"] = estabelecimentos.CNES.apply(lambda x: len(logradouros[str(x)]))


In [13]:
def differenciate_tokens(str1: str, str2: str):
    if len(str1) > len(str2):
        return differenciate_tokens(str2, str1)
    else:
        tokens1 = set(remove_accents(str1.lower()).split(" "))
        tokens2 = set(remove_accents(str2.lower()).split(" "))
        return tokens2 - tokens1

In [14]:
def palavra_diferente(row):
    if isinstance(row["logradouro_from_cep"], str):
        
        return ",".join(differenciate_tokens(row["LOGRADOURO"], row["logradouro_from_cep"]))
    else:
        return None


In [15]:
estabelecimentos["diff_logr"] = estabelecimentos.apply(palavra_diferente, axis=1)

In [16]:
Counter(estabelecimentos.loc[~estabelecimentos.diff_logr
.apply(lambda x: "," in x if isinstance(x,str) else False),"diff_logr"]
).most_common()

[('', 4971),
 ('rua', 3106),
 ('avenida', 966),
 ('doutor', 121),
 ('coronel', 78),
 ('marechal', 63),
 ('rodovia', 57),
 ('prefeito', 36),
 ('quinze', 27),
 ('sete', 26),
 ('humaita', 25),
 ('travessa', 22),
 ('15', 21),
 ('de', 18),
 ('governador', 16),
 ('schimidt', 16),
 ('praca', 14),
 ('presidente', 14),
 ('sandrescky', 14),
 ('estrada', 14),
 ('professor', 11),
 ('alameda', 11),
 ('lucca', 10),
 ('vereador', 10),
 ('772', 9),
 ('araci', 8),
 ('junior', 8),
 ('independencia', 7),
 ('dois', 7),
 ('dr', 7),
 ('148', 7),
 ('peixoto', 7),
 ('virissimo', 7),
 ('servidao', 6),
 ('doehler', 6),
 ('70', 6),
 ('professora', 6),
 ('arthur', 6),
 ('victor', 6),
 ('kubistschek', 6),
 ('planincheck', 6),
 ('19', 6),
 ('osvaldo', 5),
 ('da', 5),
 ('padre', 5),
 ('schmidt', 5),
 ('manoel', 5),
 ('jose', 5),
 ('izabel', 5),
 ('general', 5),
 ('meirelles', 4),
 ('ruy', 4),
 ('benjamin', 4),
 ('ammon', 4),
 ("d'eca", 4),
 ('698', 4),
 ('paraiba', 4),
 ('blumenau', 4),
 ('4a', 4),
 ('deputado', 4),

# Comparando dicionários

In [17]:
# Ficou ruim
def compare_addr_dict(dict1, dict2) -> dict:
    """Para cada um dos critérios abaixo, aplica uma função correspondente que retorna um 
    valor entre 0 e 1, em que 1 representa que as duas strings são idênticas."""
    
    igual = lambda x, y: 1.0 if x == y else 0.0
    
    # se forem iguais retorna 1, se não retornam 
    # o quantos % as strings são parecidas 
    def parecida(x: str, y: str):
        if len(x) > len(y):
            return parecida(y,x)
        return 1 - levenshteinDistanceDP(x,y)/len(y)  

    funcoes_comparacao = {
        "n": igual,
        "logradouro": parecida,
        "bairro": igual, 
        "municipio": igual, 
        "CEP": parecida,
        "CEPhead": parecida
    }
    dicionario_comparativo = {}
    for crit, funcao in funcoes_comparacao.items(): 
        try:
            string1, string2 = (remove_accents(x.lower()) for x in (dict1[crit], dict2[crit]))
            dicionario_comparativo[crit] = funcao(string1,string2) 
        except KeyError:
            print(f"Não possui {crit}")

    return dicionario_comparativo



def search_address(cnes: int):
    endereco_registrado = get_endereco(cnes)
    lista_osm = logradouros_list(str(cnes))
    return [compare_addr_dict(endereco_registrado, x) for x in lista_osm] 

## Limpando `display_name`

In [18]:
def sanitize_display_name(display_name: str) -> str:
    regiao1 = r"Região Geográfica Imediata de (.*?)*,"
    regiao2 = r"Região Geográfica Intermediária de (.*?),"
    texto = remove_regex(regiao1, display_name)
    texto = remove_regex(regiao2, texto)
    return remove_accents(texto.lower())



# Vamos tentar outro _approach_

Vamos agrupar os endereços por CEP e número. 
Para cada CEP e número, vamos tentar verificar se o logradouro é o mesmo. 

In [19]:
def logradouro_from_cep_numero(string: str) -> list:
    cep, numero = string.split("|")
    return estabelecimentos.query(f"CEP == {cep} and NUMERO == '{numero}'")["LOGRADOURO"].values

In [20]:
estabelecimentos["cepnumero"] = estabelecimentos.apply(lambda x: f"{x.CEP}|{x.NUMERO}", axis=1)

In [21]:
def encontrar_pertinencias(lista: list) -> list:
    __pertinencia = np.array([[a in b for a  in lista ] for b in lista ])
    try: 
        return [ (a,b) for (a, b) in zip(*np.where(__pertinencia)) if a != b]
    except ValueError:
        return []


In [22]:
class MultiCounter():
    def __init__(self):
        self.__dict__ = {}

    def append(self,key,value):
        if key in self.__dict__:
            self.__dict__[key].append(value)
        else:
            self.__dict__[key] = [value]
    def __repr__(self):
        return self.__dict__

    def counter(self):
        return {key: Counter(value).most_common() for (key, value) in self.__dict__.items()}
            

In [23]:
variacoes_cep = []
variacoes_logradouro = MultiCounter()

for cepn in estabelecimentos.cepnumero.unique():
    conjunto = list(set(logradouro_from_cep_numero(cepn)))
    nn = len(conjunto)
    variacoes_cep.append(nn)
    if nn > 1:
        for a,b in encontrar_pertinencias(conjunto):
            grande = conjunto[a]
            pequena = conjunto[b]
            resultado  = grande.replace(pequena,"")
            variacoes_logradouro.append(1,resultado)


## Rankeando o `display_name`

Cada CNES possui uma lista de resultados obtidos pelo Nominatin. 
Classificamos os resultados de cada resultado conforme contenha o 
- municipio
- bairro
- logradouro
- número
- CEP

Após isso, pegamos índice e valor do rank com a função `get_higherrank_index`

In [24]:

# 9285571
def rank_display_name(cnes: int):
    dicionario  = get_endereco(cnes)
    lista_endereços = logradouros[str(cnes)] 
    RANKS = []
    for x in lista_endereços:
        dname = sanitize_display_name(x["display_name"])
        order = ['municipio', 'bairro', 'logradouro', 'n','CEPhead', 'CEP', 'cep_logr' ]
        comparativos = [dname, dname, dname, remove_regex(r"\d{5}-\d{3}", dname), dname, dname, dname]
        rank = np.array([
        remove_accents(dicionario[x].lower()) in y for (x,y) in zip(order,comparativos)
        ])
        pesos = np.array([64,32,16,8,4,2,1])
        score = rank@pesos 
        RANKS.append(score)

    return RANKS

In [25]:
def get_higherrank_index(row):
    try:
        ranks = rank_display_name(row["CNES"])
        if len(ranks) > 0:
            i = np.argmax(ranks)
            return [i, ranks[i]]
        else:
            return [-1,0]
    except:
        return [-1,0]



def get_higherrank_info(row, info: str): 
    if row["osm_index"] > -1:  
        return logradouros[str(row["CNES"])][row["osm_index"]][info]
    else: 
        retornos = {"lat": 0.0, "lon": 0.0, "display_name":"AttributeError", "rank": 0 }
        return retornos[info]

In [26]:
estabelecimentos[["osm_index", "osm_rank"]] = estabelecimentos.apply(get_higherrank_index,axis=1,result_type="expand")

In [27]:
lat_osm = lambda x: float(get_higherrank_info(x,"lat"))
lon_osm = lambda x: float(get_higherrank_info(x,"lon"))
dname_osm = lambda x: get_higherrank_info(x,"display_name") 

estabelecimentos["lat_osm"] = estabelecimentos.apply(lat_osm,axis=1)
estabelecimentos["lon_osm"] = estabelecimentos.apply(lon_osm,axis=1)
estabelecimentos["display_name"] = estabelecimentos.apply(dname_osm,axis=1) 

    

In [29]:
differenciate_tokens(*estabelecimentos.loc[9285571,["LOGRADOURO", "logradouro_from_cep"]].values)

{'rua', 'schmidt'}

In [30]:
def nthDigit(a, n, b):
 
    # Skip N-1 Digits in Base B
    for i in range(1, n):
        a = a // b
 
    # Nth Digit from right in Base B
    return a % b

# Critérios

* `rank_osm >= 100` indica que o endereço concorda com o OSM em municipio, bairro e CEPhead
* `osm_rank > 66 and osm_rank % 2 == 1` indica que o enderço concorda em municipio, CEP e cep_from_logradouro
* `osm_rank == 68` indica que bate o municipio e o logradouro


In [31]:
estabelecimentos["confiabilidade"] = 0.0

In [32]:
indice1  = estabelecimentos.query("osm_rank >= 100 or (osm_rank > 66 and osm_rank % 2 == 1)").index
indice2 = estabelecimentos.query("osm_rank >= 80  and osm_rank < 100 ").index
indice3 = estabelecimentos.query("osm_rank >= 68 ").index

estabelecimentos.loc[indice1, "confiabilidade"] = 1.0
estabelecimentos.loc[indice2, "confiabilidade"] = 0.9

estabelecimentos.loc[indice3, "confiabilidade"] = 0.75



In [33]:
index4 = estabelecimentos.query(" confiabilidade < 0.7  and logradouro_from_cep != 'not from cep' ").index
estabelecimentos.loc[index4, "confiabilidade"] = 0.5


In [34]:
def calculate_precision(row):
    lat_cep, lon_cep = row["lat_cep"], row["lon_cep"]
    lat_osm, lon_osm = row["lat_osm"], row["lon_osm"]
    if lat_cep < 0 and lat_osm < 0:
        return abs(lat_cep - lat_osm) + abs(lon_cep - lon_osm)
    else: 
        return 100_000

In [35]:
valoures =  estabelecimentos.apply(calculate_precision, axis=1).values

In [36]:
limiares = [0.01, 0.05, 0.1, 0.5, 0.8, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
for x in limiares:
    print(x, valoures[np.where(valoures < x)].shape, sep="\t")

0.01	(9346,)
0.05	(16134,)
0.1	(17228,)
0.5	(17892,)
0.8	(18016,)
1	(18061,)
2	(18102,)
3	(18109,)
4	(18109,)
5	(18124,)
6	(18124,)
7	(18124,)
8	(18124,)
9	(18124,)
10	(18124,)


# Batendo o martelo 

* Se a confiabilidade for maior do que 0.5, vamos definir lat e lon pelo OSM;
* Se for 0.5, vamos definir lat e lon pelo CEP;
* Se for menor do que 0.5, vamos definir pelas coordenadas do município;

In [37]:
with open("../data/geoloc/boundaries-raw.json") as fp:
    municipiosdbjson = json.load(fp)

In [38]:
municipiosdb = {int(key): [ float(val[0]["lat"]),   float(val[0]["lon"])] for (key, val) in municipiosdbjson.items()}

In [39]:
def final_geoloc(row): 
    confiabilidade = row["confiabilidade"]
    # se a confiabilidade for top, pega a latitude e longitude do OSM 
    if confiabilidade > 0.5: 
        return row["lat_osm"], row["lon_osm"]
    # se não, pega pelo CEP mesmo, tudo certo
    elif confiabilidade == 0.5:
        return row["lat_cep"], row["lon_cep"]
    else:
    # em último caso, vamos pegar a geoloc do município e seja o que deus quiser
        return municipiosdb[row["IBGE"]]


In [40]:
estabelecimentos[["lat","lon"]] = estabelecimentos.apply(final_geoloc, axis=1, result_type="expand")

In [41]:
uniquecepnumero = estabelecimentos.cepnumero.unique()
CEPNUMERO_DATABASE = {}
for x in uniquecepnumero:
    idx = estabelecimentos.query("cepnumero == @x")["confiabilidade"].idxmax()
    CEPNUMERO_DATABASE[x] = idx

In [44]:

for x in estabelecimentos.NUMERO.unique():
    try:
        int(x)
    except:
        print(x)

S/N
427 0
nan


In [45]:
estabelecimentos.CEP.unique().shape

(4012,)

## Logradouro

Lembrei que tem logradouro que ficou ruim mesmo. Nesse caso vamos ter que apelar.
Vou percorrer as linhas do dataframe e procurar pelo pedaço do `display_name` que tenha a menor distância do `LOGRADOURO`.
Daí é só retornar o pedaço correspondente.

In [66]:
def find_logradouro_display(test: str, string: str)-> int:
    limpa = sanitize_display_name(string)
    array = np.array([levenshteinDistanceDP(test.lower(), x) for x in limpa.split(",")])
    return  np.argmin(array)

def findlog_row(row):
    try:
        idx = find_logradouro_display(row["LOGRADOURO"], row["display_name"])
        return row["display_name"].split(",")[idx].strip()
    except TypeError:
        return "Not from OSM"

estabelecimentos["logradouro_from_osm"] = estabelecimentos.apply(findlog_row, axis=1)

cnes
488593             Avenida Centenário
552828     Rua Major Teodósio Furtado
613207             Rua Getúlio Vargas
604801        Rua Doutor Hercílio Luz
614122                         Centro
                      ...            
6792774       Avenida Santa Terezinha
6802540        Rua Ministro Calógeras
6804721            Travessa São Pedro
6781934         Rua Adelmir Pressanto
6806171             Rua Caetano Costa
Length: 21450, dtype: object

In [91]:
def final_logradouro(row):
    if row["logradouro_from_cep"] != "not from cep" and row["logradouro_from_cep"] != "AttributeError":
        return row["logradouro_from_cep"]
    else:
        if row["logradouro_from_osm"].lower() == row["BAIRRO"].lower() or row["logradouro_from_osm"] == 'AttributeError':
            return row["LOGRADOURO"].title()
        else:
            return row["logradouro_from_osm"]


estabelecimentos["logradouro"] = estabelecimentos.apply(final_logradouro, axis=1)

# Sobrescrevendo cousas

Agora vamos maximizar as infos. Para um determinado CEP e número, vamos pegar a geolo

In [100]:
estabelecimentos.query("MUNICIPIO == 'BRUSQUE' and NUMERO == '31'")[["cepnumero","logradouro","confiabilidade", "lat_osm", "lat_cep", "lat"]]

,cepnumero,logradouro,confiabilidade,lat_osm,lat_cep,lat
cnes,,,,,,
9325824,88350075|31,Rua Felipe Schmidt,0.75,-27.092180,-27.098,-27.09218
3850420,88350075|31,Rua Felipe Schmidt,0.50,0.000000,-27.098,-27.09800
2642077,88350075|31,Rua Felipe Schmidt,0.75,-27.092180,-27.098,-27.09218
515272,88350075|31,Rua Felipe Schmidt,0.75,-27.092180,-27.098,-27.09218
9991026,88350075|31,Rua Felipe Schmidt,0.75,-27.092180,-27.098,-27.09218
2641593,88350075|31,Rua Felipe Schmidt,0.75,-27.092180,-27.098,-27.09218
6279937,88350075|31,Rua Felipe Schmidt,0.75,-27.092180,-27.098,-27.09218
7927517,88350075|31,Rua Felipe Schmidt,0.75,-27.092180,-27.098,-27.09218
9612432,88350075|31,Rua Felipe Schmidt,0.75,-27.092180,-27.098,-27.09218


In [102]:
estabelecimentos.to_csv("../parsed-data/estabelecimentos-turbinado.csv")

total 19M
-rw-r--r-- 1 bossa professores 1,5M abr  7 20:18 dados.pd.pkl
-rw-r--r-- 1 bossa professores 115K abr 29 16:10 doencas2.csv
-rw-r--r-- 1 bossa professores  61K abr  7 20:18 doencas.csv
-rw-r--r-- 1 bossa professores 3,7M mai 17 16:01 estabelecimentos.pd.pkl
-rw-r--r-- 1 bossa professores  11M mai 23 11:52 estabelecimentos-turbinado.csv
-rw-r--r-- 1 bossa professores 427K abr  6 17:05 geocodes.pd.pkl
-rw-r--r-- 1 bossa professores 745K abr  7 14:08 metadados.pd.pkl
-rw-r--r-- 1 bossa professores  22K abr 29 16:10 populacao.csv
-rw-r--r-- 1 bossa professores 1,2M abr 20 15:34 saude-series.csv
